In [ ]:
import requests
import os 
import json
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from pprint import pprint 
import pytz
#import schedule
import time
from PIL import Image
from IPython.display import display


%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json
pd.options.mode.chained_assignment = None 
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, RFE, mutual_info_classif
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import asyncio
import aiohttp
import nest_asyncio




In [ ]:


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# goal_counts_df = 'C:\\Users\\Karoline Sears\\Documents\\GitHub\\hockey-streamlit\\data\\goal_counts.csv'
# goal_counts_df = pd.read_csv(goal_counts_df)
# goal_location_df = 'C:\\Users\\Karoline Sears\\Documents\\GitHub\\hockey-streamlit\\data\\goal_locations.csv'
# goal_location_df= pd.read_csv(goal_location_df)
formatted_date = "2024-04-17"
base_url = "https://api-web.nhle.com/v1/schedule/"
# api_url = f"{base_url}{formatted_date}"
api_url = "https://api-web.nhle.com/v1/schedule/2023-10-11"
response = requests.get(api_url )
content = json.loads(response.content)

In [ ]:
# #Grabbing game info from the 2023-2024 season

# # Initialize the starting and ending game IDs
# start_game_id = 2023020001
# end_game_id = 2023021307

# # Base URL for the API
# base_url = "https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId="

# # Create an empty list to store the game IDs and URLs
# game_data = []

# # Loop through the range of game IDs
# for game_id in range(start_game_id, end_game_id + 1):
#     # Create the full API URL
#     api_url = f"{base_url}{game_id}"
#     # Append the game ID and URL to the list
#     game_data.append({"game_id": game_id, "url": api_url})

# # Convert the list to a DataFrame
# game_df = pd.DataFrame(game_data)
# game_df.head()

In [ ]:
# url = "https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId=2023020001"
#url = "https://api-web.nhle.com/v1/gamecenter/2023020260/boxscore"
#game_id = 2023021272

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # The response content can be accessed using response.text
    response_text = response.text
    #pprint(response_text)
else:
    print(f"Request failed with status code {response.status_code}")

json_data = json.loads(response_text)
# json_data.keys()
events = json_data['gameWeek']
events_df = pd.DataFrame(events)
events_df.head()
# events_df = events_df.convert_dtypes()
# details = pd.json_normalize(events_df['details'])
# details = details[['eventOwnerTeamId', 'xCoord', 'yCoord', 'shotType', 'scoringPlayerId', 'shootingPlayerId']]
# all_game_data = pd.concat([events_df[['sortOrder', 'situationCode', 'timeInPeriod', 'typeCode']], details], axis=1)

#         # Filter rows where 'typeCode' is 505-508
# shots = all_game_data[all_game_data['typeCode'].isin([505, 506, 507, 508])]
# shots['game_id']= game_id
# shots

# details = pd.DataFrame(json_data['data'])
# details.head()

In [ ]:
details['typeCode'].value_counts()

In [ ]:
details.query('typeCode == 505').head()

In [ ]:

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # The response content can be accessed using response.text
    response_text = response.text
    #pprint(response_text)
else:
    print(f"Request failed with status code {response.status_code}")

json_data = json.loads(response_text)

json_data.keys()


In [ ]:
#get the dates and game numbers for the season
game_dates = json_data['gameWeek']
game_dates

In [ ]:
#convert the game info dictionary into a dataframe that shows the date, day of week abbrev, the number of games that night and a nested table of games
df_game_info= pd.DataFrame(game_dates)
df_game_info = df_game_info[df_game_info['numberOfGames'] != 0]
#df_game_info = df_game_info.convert_dtypes()
df_game_info


In [ ]:
#Normalize (separates the data in the curly brackets out into separate columns) the data in the games column to see the games
#creates a table where every row is each date from the week and each cell contains game info
game_data =pd.json_normalize(df_game_info['games'])
game_data= pd.DataFrame(game_data)
game_data.head()
# #end_value = len(game_data.columns)


In [ ]:
#in order to extract the data properly, it needs to be stored in a dictionary. This extracts each of the games from the cells

dfs = {}

# Loop through the iterations (30 times)
for i in range(0, len(game_data.columns)): 
    api_response = game_dates
    
    if api_response is not None:
        # Extract relevant data from the API response and normalize it
        game_info = pd.json_normalize(game_data[i])
        
        # Create a DataFrame for this iteration
        df_name = f'game_test{i}'  # Generate a unique variable name
        dfs[df_name] = pd.DataFrame(game_info)
    else:
        # Handle the case where the API request failed
        print(f"API request failed for index {i}")


In [ ]:
#Then I combine all of the dfs in the list by concatenation to create a single df. now all of the game data is spread out across each row. 
combined_df = pd.concat(dfs.values(), ignore_index=True)
combined_df.head()

In [ ]:
combined_df.dropna(how='all', inplace=True)


In [ ]:
combined_df['id']

In [ ]:


combined_df = combined_df[['id', 'season', 'startTimeUTC', 'gameType', 'awayTeam.id', 'awayTeam.abbrev', 'awayTeam.logo', 
                        'homeTeam.id', 'homeTeam.abbrev','homeTeam.logo', 'homeTeam.placeName.default', 'awayTeam.placeName.default',
                        'awayTeam.score', 'homeTeam.score', 'winningGoalScorer.playerId', 
                        'winningGoalie.playerId', 'gameState']]
combined_df=combined_df.convert_dtypes()
combined_df['id'] = combined_df['id'].astype(str)
combined_df['link_prefix'] = 'https://api-web.nhle.com/v1/gamecenter/'
combined_df['link_suffix'] = '/play-by-play'
combined_df['link']=combined_df['link_prefix'] + combined_df['id'] + combined_df['link_suffix']

# Assuming '<NA>' is a string, replace it with np.nan
combined_df['id'] = combined_df['id'].replace('<NA>', np.nan)

# Drop rows with NaN values in the 'link' column
combined_df = combined_df.dropna(subset=['id'])
combined_df = combined_df.query('gameState == "OFF"')
combined_df['startTimeUTC'] = pd.to_datetime(combined_df['startTimeUTC'])
combined_df = combined_df.rename(columns = {'id':'game_id'})
combined_df.head()

In [ ]:
combined_df.head()

In [ ]:


# Specify the UTC time zone
utc_timezone = pytz.utc

# Specify the target time zone (Eastern Time)
eastern_timezone = pytz.timezone('America/New_York')

# Convert 'startTimeUTC' to Eastern Time
combined_df['game_date'] = combined_df['startTimeUTC'].dt.tz_convert(eastern_timezone)
combined_df['game_date'] = pd.to_datetime(combined_df['game_date'])
combined_df['game_date'] = combined_df['game_date'].dt.strftime('%Y-%m-%d')
combined_df.drop('startTimeUTC', axis=1, inplace=True)
# combined_df = combined_df[combined_df['game_date'] == formatted_date]

# print("combined_df done")
combined_df.head()

In [ ]:
combined_df.sort_values(by='game_id')

In [ ]:

home_df = pd.DataFrame({
    'game_id': combined_df['game_id'],
    'team_id': combined_df['homeTeam.id'],
    'tri_code': combined_df['homeTeam.abbrev'],
    'team_name': combined_df['homeTeam.placeName.default'],
    'value': "home",
    'team_logo': combined_df['homeTeam.logo']
})

away_df =  pd.DataFrame({
    'game_id': combined_df['game_id'],
    'team_id': combined_df['awayTeam.id'],
    'tri_code': combined_df['awayTeam.abbrev'],
    'team_name': combined_df['awayTeam.placeName.default'],
    'value':"away",
    'team_logo':combined_df['awayTeam.logo']
})    


In [ ]:

#Roster Data
team_url = "https://api.nhle.com/stats/rest/en/team"

response = requests.get(team_url )
content = json.loads(response.content)

# Send an HTTP GET request to the specified URL
response = requests.get(team_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # The response content can be accessed using response.text
    response_text = response.text
    #pprint(response_text)
else:
    print(f"Request failed with status code {response.status_code}")

json_data = json.loads(response_text)
# json_data.keys()

roster = json_data['data']


In [ ]:
roster

In [ ]:


df_roster= pd.DataFrame(roster)
df_roster = df_roster.convert_dtypes()
df_roster = df_roster[['id', 'fullName', 'triCode']]
df_roster['prefix'] = "https://api-web.nhle.com/v1/roster/"
df_roster['suffix'] = "/20232024"
df_roster['roster_url'] = df_roster['prefix'] + df_roster['triCode'] + df_roster['suffix']
df_roster = df_roster[['id','fullName', 'triCode', 'roster_url']]
df_roster = df_roster.rename(columns = {'id':'team_id', 'fullName':'team_name', 'triCode':'tri_code'})
df_roster=df_roster.sort_values(by='team_id')


valid_team_codes = set(range(1, 11)).union(set(range(12, 27))).union(set(range(28, 31))).union(set(range(52, 53))).union(set(range(54, 56))).union(set(range(59, 60)))
filtered_rosters = df_roster[df_roster['team_id'].isin(valid_team_codes)]
filtered_rosters



In [ ]:
filtered_rosters.head()

In [ ]:

# Create an empty dictionary to store the results
roster_dict = {}
roster_link = filtered_rosters['roster_url'] 

    # Define your extraction script as a function
def extract_roster_data(roster_link):
    try:
        # Send an HTTP GET request to the game URL
        response = requests.get(roster_link)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            response_text = response.text
            json_data = json.loads(response_text)

            forwards = json_data['forwards']
            forwards_df = pd.DataFrame(forwards)
            forwards_df = forwards_df.convert_dtypes()
            forwards_df = forwards_df[['id', 'headshot', 'firstName', 'lastName', 'sweaterNumber', 'positionCode']]
            forwards_df = forwards_df.dropna(subset=['sweaterNumber'])

            # Normalize (separates the data in the curly brackets out into separate columns)
            forwards_last = pd.json_normalize(forwards_df['lastName'])
            forwards_last = pd.DataFrame(forwards_last)
            forwards_last = forwards_last[['default']]
            forwards_last = forwards_last.rename(columns={'default': 'last_name'})

            forwards_first = pd.json_normalize(forwards_df['firstName'])
            forwards_first = pd.DataFrame(forwards_first)
            forwards_first = forwards_first[['default']]
            forwards_first = forwards_first.rename(columns={'default': 'first_name'})

            forwards_roster = forwards_df.merge(forwards_first, left_index=True, right_index=True).merge(forwards_last, left_index=True, right_index=True)
            forwards_roster['player_name'] = forwards_roster['first_name'] + " " + forwards_roster['last_name']
            forwards_roster = forwards_roster[['id', 'headshot', 'player_name', 'sweaterNumber', 'positionCode']]

            defense = json_data['defensemen']
            defense_df = pd.DataFrame(defense)
            defense_df = defense_df.convert_dtypes()
            defense_df = defense_df[['id', 'headshot', 'firstName', 'lastName', 'sweaterNumber', 'positionCode']]
            defense_df = defense_df.dropna(subset=['sweaterNumber'])

            # Normalize (separates the data in the curly brackets out into separate columns)
            defense_last = pd.json_normalize(defense_df['lastName'])
            defense_last = pd.DataFrame(defense_last)
            defense_last = defense_last[['default']]
            defense_last = defense_last.rename(columns={'default': 'last_name'})

            defense_first = pd.json_normalize(defense_df['firstName'])
            defense_first = pd.DataFrame(defense_first)
            defense_first = defense_first[['default']]
            defense_first = defense_first.rename(columns={'default': 'first_name'})

            defense_roster = defense_df.merge(defense_first, left_index=True, right_index=True).merge(defense_last, left_index=True, right_index=True)
            defense_roster['player_name'] = defense_roster['first_name'] + " " + defense_roster['last_name']
            defense_roster = defense_roster[['id', 'headshot', 'player_name', 'sweaterNumber', 'positionCode']]

            goalies = json_data['goalies']
            goalies_df = pd.DataFrame(goalies)
            goalies_df = goalies_df.convert_dtypes()
            goalies_df = goalies_df[['id', 'headshot', 'firstName', 'lastName', 'sweaterNumber', 'positionCode']]
            goalies_df = goalies_df.dropna(subset=['sweaterNumber'])

            # Normalize (separates the data in the curly brackets out into separate columns)
            goalies_last = pd.json_normalize(goalies_df['lastName'])
            goalies_last = pd.DataFrame(goalies_last)
            goalies_last = goalies_last[['default']]
            goalies_last = goalies_last.rename(columns={'default': 'last_name'})

            goalies_first = pd.json_normalize(goalies_df['firstName'])
            goalies_first = pd.DataFrame(goalies_first)
            goalies_first = goalies_first[['default']]
            goalies_first = goalies_first.rename(columns={'default': 'first_name'})

            goalie_roster = goalies_df.merge(goalies_first, left_index=True, right_index=True).merge(goalies_last, left_index=True, right_index=True)
            goalie_roster['player_name'] = goalie_roster['first_name'] + " " + goalie_roster['last_name']
            goalie_roster = goalie_roster[['id', 'headshot', 'player_name', 'sweaterNumber', 'positionCode']]

            # Append all data to form a single team roster
            team_roster_df = pd.concat([forwards_roster, defense_roster, goalie_roster], axis=0, ignore_index=True)

            team_roster_df['id'] = team_roster_df['id'] .astype(float, errors='ignore')

                                
            def extract_tricode(link):
                # Use a regular expression to find the tricode
                match = re.search(r'/([A-Z]{3})/', link)
                if match:
                    return match.group(1)
                else:
                    # Return None if no match is found
                    return None

            team_roster_df['tri_code'] = team_roster_df['headshot'].apply(extract_tricode)
            return team_roster_df  # Return the entire processed DataFrame
        else:
            print(f"Request failed for {roster_link} with status code {response.status_code}")
            return None  # Return None to indicate failure
    
    except Exception as e:
            print(f"An error occurred: {e}")
            return None  # Return None to indicate failure
    
# Loop through the rows of the filtered_rosters df
for index, row in filtered_rosters.iterrows():
    # Extract the API link from the current row
    roster_link = row['roster_url']

    # Run your game-specific data script and get the entire processed DataFrame
    team_roster = extract_roster_data(roster_link)
    
    # Check if team_roster is not None before proceeding
    if team_roster is not None:
        # Add a 'game_id' column to the game_specific_data DataFrame
        team_roster['tri_code'] = row['tri_code']

        # Store the result in the dictionary with the game ID as the key
        roster_dict[row['tri_code']] = team_roster
    else:
        print(f"Skipping row {index} due to failed request or exception.")

all_rosters = pd.concat(roster_dict.values(), ignore_index=True)

team_rosters = filtered_rosters[['team_id', 'team_name', 'tri_code']]
team_rosters = team_rosters.merge(all_rosters,  on = "tri_code", how = "left")
team_rosters = team_rosters.rename(columns= {'id':'player_id'})


print("rosters done")


In [ ]:

games = combined_df[['game_id', 'season', 'gameType', 'awayTeam.id', 'awayTeam.abbrev', 'awayTeam.logo', 
                        'homeTeam.id', 'homeTeam.abbrev','homeTeam.logo', 'awayTeam.score', 'homeTeam.score', 'winningGoalScorer.playerId', 
                        'winningGoalie.playerId', 'link']]
games = games.sort_values(by='game_id')
games = games.rename(columns = {'gameType':'game_type', 'awayTeam.id':'away_team_id', 'awayTeam.abbrev':'away_team_tricode', 
                    'awayTeam.logo': 'away_logo', 'homeTeam.id':'home_team_id', 'homeTeam.abbrev':'home_team_tricode', 'homeTeam.logo':'home_logo', 
                    'awayTeam.score': 'away_score', 'homeTeam.score':'home_score', 'winningGoalScorer.playerId':'winning_goal_player_id', 
                    'winningGoalie.playerId':'winning_goalie_id'})


games = pd.merge(games, df_roster, left_on='home_team_id', right_on='team_id', how='left')

# Rename columns and drop redundant columns
games.rename(columns={'roster_url': 'home_roster_link', 'team_name':'home_team_name'}, inplace=True)
games.drop(columns=['team_id', 'tri_code'], inplace=True)

# Merge DataFrames based on away_team_code
games = pd.merge(games, df_roster, left_on='away_team_id', right_on='team_id', how='left')

# Rename columns and drop redundant columns
games.rename(columns={'roster_url': 'away_roster_link', 'team_name':'away_team_name'}, inplace=True)
games.drop(columns=['team_id','tri_code'], inplace=True)

# Assuming '<NA>' is a string, replace it with np.nan
games['game_id'].replace('<NA>', np.nan, inplace=True)
# Drop rows with NaN values in the 'link' column
games = games.dropna(subset=['game_id'])

# Reset index after dropping rows
games.reset_index(drop=True, inplace=True)


In [ ]:
games.head()

In [ ]:

# Create an empty dictionary to store the results
for index, row in games.iterrows():
    # Check if 'link' is in the columns of the current row
    if 'link' not in row.index:
        print(f"Skipping row {index} as it does not contain 'link' column.")
        continue

game_link = row['link']
game_data_dict = {}
game_id = row['game_id']

    # Define your extraction script as a function
def extract_game_data(game_link):
    try:
        # Send an HTTP GET request to the game URL
        response = requests.get(game_link)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            response_text = response.text
            json_game_data = json.loads(response_text)

            #event data
            event_data = json_game_data['plays']
            events_df = pd.DataFrame(event_data)
            events_df = events_df.convert_dtypes()

            #game events data
            game_events = events_df[['sortOrder','timeInPeriod', 'homeTeamDefendingSide', 'typeCode']]

            #coordinates data
            details = pd.json_normalize(events_df['details'])
            details = pd.DataFrame(details)
            details = details[['eventOwnerTeamId', 'xCoord', 'yCoord', 'shotType', 'scoringPlayerId']]

            #join the dfs
            all_game_data = game_events.merge(details, left_index=True, right_index=True)

            # Filter rows where 'typeCode' is 505
            shots = all_game_data[all_game_data['typeCode'].isin([505, 506, 507, 508])]

            # Convert the 'player_id' column to the appropriate data type (e.g., int)
            #all_game_data['scoringPlayerId'] = all_game_data['scoringPlayerId'].astype(float).astype(pd.Int64Dtype(), errors='ignore')

            # Create a 'game_id' column
            all_game_data['game_id'] = row['game_id']

            return shots  # Return the processed DataFrame and game_id

        else:
            print(f"Request failed for {game_link} with status code {response.status_code}")
            return pd.DataFrame(), None  # Return an empty DataFrame and None to indicate failure

    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame(), None  # Return an empty DataFrame and None to indicate failure
    
    # Run your game-specific data script and get the entire processed DataFrame and game_id
for index, row in games.iterrows():
    # Extract the API link from the current row
    game_link = row['link']
    
# Run your game-specific data script and get the entire processed DataFrame
    game_specific_data = extract_game_data(game_link)

        # Add a 'game_id' column to the game_specific_data DataFrame
    game_specific_data['game_id'] = row['game_id']
    
    # Store the result in the dictionary with the game ID as the key
    game_data_dict[row['game_id']] = game_specific_data

game_data = pd.concat(game_data_dict.values(), ignore_index=True)
game_goals_data = game_data[game_data['typeCode'] == 505]
game_goals_data = game_goals_data.rename(columns = {'scoringPlayerId':'player_id'})

team_rosters = team_rosters.rename(columns = {'sweaterNumber': 'jersey_number', 'positionCode':'position'})

print("goals done")


In [ ]:
game_data

In [ ]:
game_specific_data.head()

In [ ]:

shots_df = game_specific_data[game_specific_data['typeCode'].isin([505, 506, 507, 508])]
shots_df = shots_df.sort_values(by=['timeInPeriod', 'sortOrder'])

shots_df=shots_df.rename(columns = {'eventOwnerTeamId':'team_id'})
shots_df = shots_df.dropna(subset=['team_id'])
shots_df['team_id'] =shots_df['team_id'].astype('Int64').astype(int)


In [ ]:
shots_df.head()

In [ ]:

#Merging game data and roster data to create goal counts
if 'player_id' in team_rosters.columns:
    # Merge the coordinates data from final_game_data to goal_counts based on player IDs
    goal_counts = team_rosters.merge(game_goals_data[['player_id', 'sortOrder']],on = 'player_id')
else:
    print("Column 'player_id 'does not exist in goal_counts DataFrame.")

goal_counts = goal_counts.groupby(['jersey_number','player_id', 'player_name', 'position', 'team_name'])['sortOrder'].count()
goal_counts=pd.DataFrame(goal_counts).reset_index()
goal_counts = goal_counts.rename(columns={'sortOrder': 'goals'})
goal_counts = goal_counts.sort_values(by='player_name')
goal_counts.head()

In [ ]:
#game_goals_data

In [ ]:

if 'player_id' in team_rosters.columns:
    # Merge the coordinates data from final_game_data to goal_counts based on player IDs
    goal_locations = team_rosters.merge(game_goals_data[['player_id', 'sortOrder', 'xCoord','yCoord', 'game_id']],on = 'player_id')
else:
    print("Column 'player_id 'does not exist in goal_counts DataFrame.")

goal_locations = goal_locations[['jersey_number', 'player_id','player_name', 'position', 'team_name', 'sortOrder', 'xCoord', 'yCoord', 'game_id']]
goal_locations = goal_locations.dropna(subset=['player_id'])
goal_locations = goal_locations.sort_values(['player_name', 'player_id', 'sortOrder'], ascending=[True, True, True])
goal_locations.head()

In [ ]:

#append the data to the master set
goal_location_df = pd.concat([goal_location_df, goal_locations], ignore_index=True)
goal_location_df =  goal_location_df.sort_values(['player_name', 'player_id', 'sortOrder'], ascending=[True, True, True])

# file_path1 = 'C:\\Users\\Karoline Sears\\Documents\\GitHub\\hockey-streamlit\\data\\goal_locations.csv'
# goal_location_df.to_csv(file_path1,  index=False, encoding='utf-8')

# Append last_nights_goals to season_goals
combined_goals = pd.concat([goal_counts_df, goal_counts], ignore_index=True)

# Sum up the goals for duplicated players
result = combined_goals.groupby(['jersey_number', 'player_id', 'player_name', 'position', 'team_name'], as_index=False)['goals'].sum()
result=result.sort_values(by='player_name')

file_path2 = 'C:\\Users\\Karoline Sears\\Documents\\GitHub\\hockey-streamlit\\data\\goal_counts.csv'
result.to_csv(file_path2,  index=False, encoding='utf-8')


In [ ]:

#running count of goal number for each player - make sure this is ordered by game id and time in period

goal_talley =goal_location_df.sort_values(['player_name', 'player_id', 'game_id', 'sortOrder', 'period'], 
                                          ascending=[True, True, True, True, True])
goal_talley['goal_no'] = goal_talley.groupby('player_id').cumcount()+1

def adjust_coordinates(row):
    x = row['xCoord']
    y = row['yCoord']
    if x < 0:
        adj_x = abs(x)
        adj_y = -y
    else:
        adj_x = x
        adj_y = y
    return pd.Series({'x_adjusted': adj_x, 'y_adjusted': adj_y})

# Apply the function to each row of the DataFrame
goal_talley[['x_adjusted', 'y_adjusted']] = goal_talley.apply(adjust_coordinates, axis=1)
ice_map_data = goal_talley
ice_map_data=ice_map_data[['game_id', 'goal_no', 'player_id','player_name', 'team_name', 'period', 'x_adjusted', 'y_adjusted' ]]

# file_path3 = 'C:\\Users\\Karoline Sears\\Documents\\GitHub\\hockey-streamlit\\data\\ice_map_data.csv'
# ice_map_data.to_csv(file_path3,  index=False, encoding='utf-8')
print("completed", formatted_date)